# Projeto Final — Camada Silver
### Análise de Reclamações Financeiras (Agibank)

**Objetivo:** identificar colunas necessárias à análise e aquelas que podem ser inutilizadas.

Etapas:
1. Carregamento da base Bronze consolidada  
2. Diagnóstico inicial dos dados  
3. Pesquisa técnica das colunas administrativas  

In [38]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import os
from pathlib import Path

plt.style.use("seaborn-v0_8-whitegrid")
sns.set_context("talk")
sns.set_palette(["#0064F5", "#77DF40", "#FFD600"])

In [39]:
arquivo_bronze = "../../data/silver/consumidor_gov_bronze.csv"

df = pd.read_csv(arquivo_bronze, sep=";", encoding="utf-8", low_memory=False)

print("Formato da base:", df.shape)
display(df.head())

Formato da base: (2567429, 35)


,Gestor,Canal de Origem,Região,UF,Cidade,Sexo,Faixa Etária,Ano Abertura,Mês Abertura,Data Abertura,...,Nota do Consumidor,Análise da Recusa,data_source,file_origin,processed_at,file_month,is_agibank,quality_score,Interação com Judiciario,Último Complemento Consumidor
0,Programa Estadual de Proteção e Defesa do Cons...,Plataforma Web,SE,MG,Belo Horizonte,M,entre 61 a 70 anos,2024,11,01/11/2024,...,1.0,NaN,consumidor_gov,basecompleta2025-01.csv,2026-02-01 23:08:30.444138,01/2025,False,0.9,NaN,NaN
1,Secretaria Adjunta de Proteção e Defesa dos Di...,Plataforma Web,CO,MT,Cuiabá,M,entre 31 a 40 anos,2024,11,01/11/2024,...,1.0,NaN,consumidor_gov,basecompleta2025-01.csv,2026-02-01 23:08:30.444138,01/2025,False,0.9,NaN,NaN
2,Secretaria Nacional do Consumidor,Plataforma Web,NE,SE,Aracaju,M,entre 21 a 30 anos,2024,11,01/11/2024,...,5.0,NaN,consumidor_gov,basecompleta2025-01.csv,2026-02-01 23:08:30.444138,01/2025,False,0.9,NaN,NaN
3,Secretaria Nacional do Consumidor,Plataforma Web,CO,MS,Campo Grande,F,entre 61 a 70 anos,2024,11,01/11/2024,...,5.0,NaN,consumidor_gov,basecompleta2025-01.csv,2026-02-01 23:08:30.444138,01/2025,False,0.9,NaN,NaN
4,Instituto Municipal de Proteção e Defesa do Co...,Plataforma Web,SE,RJ,Rio de Janeiro,M,entre 41 a 50 anos,2024,11,01/11/2024,...,3.0,NaN,consumidor_gov,basecompleta2025-01.csv,2026-02-01 23:08:30.444138,01/2025,False,0.9,NaN,NaN


In [40]:
resumo_geral = pd.DataFrame({
    "Tipo": df.dtypes,
    "Nº de Valores Únicos": df.nunique(dropna=True),
    "Nº de Nulos": df.isna().sum(),
    "% de Nulos": (df.isna().mean()*100).round(2)
}).reset_index()

resumo_geral.columns = ["Coluna", "Tipo", "Nº de Valores Únicos", "Nº de Nulos", "% de Nulos"]

display(resumo_geral)

,Coluna,Tipo,Nº de Valores Únicos,Nº de Nulos,% de Nulos
0,Gestor,object,17,0,0.00
1,Canal de Origem,object,1,0,0.00
2,Região,object,5,0,0.00
3,UF,object,27,0,0.00
4,Cidade,object,6633,0,0.00
5,Sexo,object,3,55,0.00
6,Faixa Etária,object,7,0,0.00
7,Ano Abertura,int64,2,0,0.00
8,Mês Abertura,int64,12,0,0.00
9,Data Abertura,object,425,0,0.00


In [41]:
candidatas_exclusao = resumo_geral[
    (resumo_geral["% de Nulos"] > 80) | (resumo_geral["Nº de Valores Únicos"] <= 3)
]

candidatas_exclusao = candidatas_exclusao.sort_values(by="% de Nulos", ascending=False)

display(candidatas_exclusao)

,Coluna,Tipo,Nº de Valores Únicos,Nº de Nulos,% de Nulos
34,Último Complemento Consumidor,object,72,2472844,96.32
33,Interação com Judiciario,object,2,2300055,89.59
26,Análise da Recusa,object,3,2238356,87.18
24,Avaliação Reclamação,object,3,261708,10.19
21,Procurou Empresa,object,2,0,0.00
7,Ano Abertura,int64,2,0,0.00
5,Sexo,object,3,55,0.00
1,Canal de Origem,object,1,0,0.00
27,data_source,object,1,0,0.00
22,Respondida,object,2,0,0.00


In [42]:
filtro_1 = resumo_geral[
    (resumo_geral["% de Nulos"] > 80) & 
    (resumo_geral["Nº de Valores Únicos"] < 10)
]

print("Colunas com > 80% nulos E < 10 valores únicos:")
display(filtro_1.sort_values(by="% de Nulos", ascending=False))

Colunas com > 80% nulos E < 10 valores únicos:


,Coluna,Tipo,Nº de Valores Únicos,Nº de Nulos,% de Nulos
33,Interação com Judiciario,object,2,2300055,89.59
26,Análise da Recusa,object,3,2238356,87.18


In [43]:
filtro_2 = resumo_geral[resumo_geral["Nº de Valores Únicos"] == 1]

print("Colunas com apenas 1 valor único (fixas):")
display(filtro_2)

Colunas com apenas 1 valor único (fixas):


,Coluna,Tipo,Nº de Valores Únicos,Nº de Nulos,% de Nulos
1,Canal de Origem,object,1,0,0.0
27,data_source,object,1,0,0.0
32,quality_score,float64,1,0,0.0


In [44]:
colunas_filtro_1 = filtro_1["Coluna"].tolist()

print("Amostra das colunas do Filtro 1:")
display(df[colunas_filtro_1].head(10))

Amostra das colunas do Filtro 1:


,Análise da Recusa,Interação com Judiciario
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [45]:
filtro_4 = resumo_geral[
    (resumo_geral["Nº de Valores Únicos"] >= 2) & 
    (resumo_geral["Nº de Valores Únicos"] <= 3)
]

print("Colunas com 2 ou 3 valores únicos:")
display(filtro_4)

Colunas com 2 ou 3 valores únicos:


,Coluna,Tipo,Nº de Valores Únicos,Nº de Nulos,% de Nulos
5,Sexo,object,3,55,0.00
7,Ano Abertura,int64,2,0,0.00
21,Procurou Empresa,object,2,0,0.00
22,Respondida,object,2,0,0.00
24,Avaliação Reclamação,object,3,261708,10.19
26,Análise da Recusa,object,3,2238356,87.18
31,is_agibank,bool,2,0,0.00
33,Interação com Judiciario,object,2,2300055,89.59


In [46]:
colunas_filtro_4 = filtro_4["Coluna"].tolist()

for col in colunas_filtro_4:
    print(f"\n{col}:")
    print(df[col].value_counts())


Sexo:
Sexo
M    1429886
F    1135633
O       1855
Name: count, dtype: int64

Ano Abertura:
Ano Abertura
2025    2458147
2024     109282
Name: count, dtype: int64

Procurou Empresa:
Procurou Empresa
S    2000174
N     567255
Name: count, dtype: int64

Respondida:
Respondida
S    2235592
N     331837
Name: count, dtype: int64

Avaliação Reclamação:
Avaliação Reclamação
Não Avaliada     1539794
Não Resolvida     430635
Resolvida         335292
Name: count, dtype: int64

Análise da Recusa:
Análise da Recusa
Procedente      218404
Improcedente     68654
Encerrada        42015
Name: count, dtype: int64

is_agibank:
is_agibank
False    2547573
True       19856
Name: count, dtype: int64

Interação com Judiciario:
Interação com Judiciario
N    267370
S         4
Name: count, dtype: int64


In [47]:
filtro_final = resumo_geral[
    (resumo_geral["% de Nulos"] > 70) & 
    (resumo_geral["Nº de Valores Únicos"] <= 5) &
    (resumo_geral["Tipo"] == "object")
]

print("Candidatos finais (alto nulo + pouca variação + texto):")
display(filtro_final.sort_values(by="% de Nulos", ascending=False))

Candidatos finais (alto nulo + pouca variação + texto):


,Coluna,Tipo,Nº de Valores Únicos,Nº de Nulos,% de Nulos
33,Interação com Judiciario,object,2,2300055,89.59
26,Análise da Recusa,object,3,2238356,87.18


In [ ]:
# Colunas que RESPONDEM perguntas ou são NECESSÁRIAS para indicadores
colunas_essenciais = [
    # Perfil do Consumidor
    "Sexo", "Faixa Etária", "Região", "UF", "Cidade",
    
    # Temporais
    "Ano Abertura", "Mês Abertura", "Data Abertura", 
    "Data Resposta", "Data Finalização", "Tempo Resposta",
    
    # Instituições
    "Nome Fantasia", "Segmento de Mercado", "Área", "is_agibank",
    
    # Reclamações
    "Assunto", "Grupo Problema", "Problema",
    
    # Comportamento
    "Canal de Origem", "Como Comprou Contratou", "Procurou Empresa",
    
    # Satisfação e Resolução
    "Respondida", "Situação", "Avaliação Reclamação", "Nota do Consumidor"
]

print(f"Colunas essenciais: {len(colunas_essenciais)}")

✅ Colunas essenciais: 25


In [49]:
todas_colunas = df.columns.tolist()
colunas_nao_essenciais = [col for col in todas_colunas if col not in colunas_essenciais]

print(f"\nColunas que NÃO estão na lista essencial ({len(colunas_nao_essenciais)}):")
for col in colunas_nao_essenciais:
    print(f"  - {col}")


Colunas que NÃO estão na lista essencial (10):
  - Gestor
  - Prazo Resposta
  - Análise da Recusa
  - data_source
  - file_origin
  - processed_at
  - file_month
  - quality_score
  - Interação com Judiciario
  - Último Complemento Consumidor


In [50]:
resumo_nao_essenciais = resumo_geral[resumo_geral["Coluna"].isin(colunas_nao_essenciais)]

print("\nResumo das colunas não essenciais:")
display(resumo_nao_essenciais.sort_values(by="% de Nulos", ascending=False))


Resumo das colunas não essenciais:


,Coluna,Tipo,Nº de Valores Únicos,Nº de Nulos,% de Nulos
34,Último Complemento Consumidor,object,72,2472844,96.32
33,Interação com Judiciario,object,2,2300055,89.59
26,Análise da Recusa,object,3,2238356,87.18
0,Gestor,object,17,0,0.00
12,Prazo Resposta,object,428,0,0.00
27,data_source,object,1,0,0.00
29,processed_at,object,12,0,0.00
28,file_origin,object,12,0,0.00
32,quality_score,float64,1,0,0.00
30,file_month,object,12,0,0.00


In [51]:
print("\nExemplos de valores:\n")
for col in colunas_nao_essenciais:
    print(f"{col}:")
    print(df[col].value_counts().head(3))
    print("-" * 60)


Exemplos de valores:

Gestor:
Gestor
Fundação de Proteção e Defesa do Consumidor             649451
Secretaria Nacional do Consumidor                       513870
Programa Estadual de Proteção e Defesa do Consumidor    300099
Name: count, dtype: int64
------------------------------------------------------------
Prazo Resposta:
Prazo Resposta
22/11/2025    18435
21/11/2025    16831
25/11/2025    14643
Name: count, dtype: int64
------------------------------------------------------------
Análise da Recusa:
Análise da Recusa
Procedente      218404
Improcedente     68654
Encerrada        42015
Name: count, dtype: int64
------------------------------------------------------------
data_source:
data_source
consumidor_gov    2567429
Name: count, dtype: int64
------------------------------------------------------------
file_origin:
file_origin
basecompleta2025-12.csv    305205
basecompleta2025-10.csv    268715
basecompleta2025-09.csv    267374
Name: count, dtype: int64
------------------------

In [52]:
regra_1 = resumo_geral[
    (resumo_geral["% de Nulos"] > 85) &
    (resumo_geral["Nº de Valores Únicos"] < 100)
]

print("Regra 1: Colunas com > 85% nulos e < 100 valores únicos\n")
display(regra_1)

print("\nExemplos de valores dessas colunas:\n")
for col in regra_1["Coluna"]:
    print(f"{col}:")
    print(df[col].value_counts().head(3))
    print("-" * 60)

Regra 1: Colunas com > 85% nulos e < 100 valores únicos



,Coluna,Tipo,Nº de Valores Únicos,Nº de Nulos,% de Nulos
26,Análise da Recusa,object,3,2238356,87.18
33,Interação com Judiciario,object,2,2300055,89.59
34,Último Complemento Consumidor,object,72,2472844,96.32



Exemplos de valores dessas colunas:

Análise da Recusa:
Análise da Recusa
Procedente      218404
Improcedente     68654
Encerrada        42015
Name: count, dtype: int64
------------------------------------------------------------
Interação com Judiciario:
Interação com Judiciario
N    267370
S         4
Name: count, dtype: int64
------------------------------------------------------------
Último Complemento Consumidor:
Último Complemento Consumidor
01/09/2025    4091
02/09/2025    4033
03/09/2025    3793
Name: count, dtype: int64
------------------------------------------------------------


In [53]:
regra_2 = resumo_geral[resumo_geral["Nº de Valores Únicos"] == 1]

print("Regra 2: Colunas com apenas 1 valor único (fixas)\n")
display(regra_2)

print("\nExemplos de valores dessas colunas:\n")
for col in regra_2["Coluna"]:
    print(f"{col}:")
    print(df[col].value_counts().head(3))
    print("-" * 60)

Regra 2: Colunas com apenas 1 valor único (fixas)



,Coluna,Tipo,Nº de Valores Únicos,Nº de Nulos,% de Nulos
1,Canal de Origem,object,1,0,0.0
27,data_source,object,1,0,0.0
32,quality_score,float64,1,0,0.0



Exemplos de valores dessas colunas:

Canal de Origem:
Canal de Origem
Plataforma Web    2567429
Name: count, dtype: int64
------------------------------------------------------------
data_source:
data_source
consumidor_gov    2567429
Name: count, dtype: int64
------------------------------------------------------------
quality_score:
quality_score
0.9    2567429
Name: count, dtype: int64
------------------------------------------------------------


In [54]:
regra_3 = resumo_geral[
    resumo_geral["Coluna"].isin(colunas_nao_essenciais) &
    (
        (resumo_geral["Tipo"] == "object") &
        (
            (resumo_geral["% de Nulos"] > 80) |
            (resumo_geral["Nº de Valores Únicos"] < 50)
        )
    )
]

print("Regra 3: Colunas não essenciais com características administrativas\n")
display(regra_3)

print("\nExemplos de valores dessas colunas:\n")
for col in regra_3["Coluna"]:
    print(f"{col}:")
    print(df[col].value_counts().head(3))
    print("-" * 60)

Regra 3: Colunas não essenciais com características administrativas



,Coluna,Tipo,Nº de Valores Únicos,Nº de Nulos,% de Nulos
0,Gestor,object,17,0,0.00
26,Análise da Recusa,object,3,2238356,87.18
27,data_source,object,1,0,0.00
28,file_origin,object,12,0,0.00
29,processed_at,object,12,0,0.00
30,file_month,object,12,0,0.00
33,Interação com Judiciario,object,2,2300055,89.59
34,Último Complemento Consumidor,object,72,2472844,96.32



Exemplos de valores dessas colunas:

Gestor:
Gestor
Fundação de Proteção e Defesa do Consumidor             649451
Secretaria Nacional do Consumidor                       513870
Programa Estadual de Proteção e Defesa do Consumidor    300099
Name: count, dtype: int64
------------------------------------------------------------
Análise da Recusa:
Análise da Recusa
Procedente      218404
Improcedente     68654
Encerrada        42015
Name: count, dtype: int64
------------------------------------------------------------
data_source:
data_source
consumidor_gov    2567429
Name: count, dtype: int64
------------------------------------------------------------
file_origin:
file_origin
basecompleta2025-12.csv    305205
basecompleta2025-10.csv    268715
basecompleta2025-09.csv    267374
Name: count, dtype: int64
------------------------------------------------------------
processed_at:
processed_at
2026-02-01 23:08:55.939625    305205
2026-02-01 23:08:49.872718    268715
2026-02-01 23:08:46.96415

In [55]:
candidatas_remocao_auto = pd.concat([regra_1, regra_2, regra_3]).drop_duplicates(subset=["Coluna"])

print("Consolidação: Todas as colunas identificadas pelas regras automáticas\n")
display(candidatas_remocao_auto.sort_values(by="% de Nulos", ascending=False))

Consolidação: Todas as colunas identificadas pelas regras automáticas



,Coluna,Tipo,Nº de Valores Únicos,Nº de Nulos,% de Nulos
34,Último Complemento Consumidor,object,72,2472844,96.32
33,Interação com Judiciario,object,2,2300055,89.59
26,Análise da Recusa,object,3,2238356,87.18
1,Canal de Origem,object,1,0,0.00
27,data_source,object,1,0,0.00
32,quality_score,float64,1,0,0.00
0,Gestor,object,17,0,0.00
28,file_origin,object,12,0,0.00
29,processed_at,object,12,0,0.00
30,file_month,object,12,0,0.00


In [56]:
metadados_manter = [
    "data_source", "file_origin", "processed_at", 
    "file_month", "quality_score"
]

candidatas_remocao_final = candidatas_remocao_auto[
    ~candidatas_remocao_auto["Coluna"].isin(metadados_manter)
]

print("Lista final (excluindo metadados):\n")
display(candidatas_remocao_final)

print("\nNomes das colunas a remover:")
colunas_remover = candidatas_remocao_final["Coluna"].tolist()
print(colunas_remover)

Lista final (excluindo metadados):



,Coluna,Tipo,Nº de Valores Únicos,Nº de Nulos,% de Nulos
26,Análise da Recusa,object,3,2238356,87.18
33,Interação com Judiciario,object,2,2300055,89.59
34,Último Complemento Consumidor,object,72,2472844,96.32
1,Canal de Origem,object,1,0,0.00
0,Gestor,object,17,0,0.00



Nomes das colunas a remover:
['Análise da Recusa', 'Interação com Judiciario', 'Último Complemento Consumidor', 'Canal de Origem', 'Gestor']
